In [16]:
import sys
import os
import numpy as np
import pandas as pd
import time
import re
import csv
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
# 웹 드라이버 초기화
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException

service = Service(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service=service)
url = "https://www.agoda.com/ko-kr/search?city=14690&checkIn=2023-12-29&los=1&rooms=1&adults=2&children=0&locale=ko-kr&ckuid=3298c6b4-c27f-4698-86c9-8a0af3e523bb&prid=0&gclid=Cj0KCQiAm4WsBhCiARIsAEJIEzX0PxKz5s9dsRAqZCCagu1R1EfBiD5SLOIvD5ME-4u28z5kjBVPAEEaAjj_EALw_wcB&currency=KRW&correlationId=8402090a-d1d7-455e-9a8f-086930b85077&analyticsSessionId=-1315171232710586634&pageTypeId=5&realLanguageId=9&languageId=9&origin=KR&cid=1891463&tag=45b17d1d-e0b0-fe2a-ce90-5513829d856b&userId=3298c6b4-c27f-4698-86c9-8a0af3e523bb&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=26&currencyCode=KRW&htmlLanguage=ko-kr&cultureInfoName=ko-kr&machineName=hk-geoweb-2004&trafficGroupId=5&sessionId=5e3l543y1jb4yppwktzzxuko&trafficSubGroupId=122&aid=82361&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&cdnDomain=agoda.net&checkOut=2023-12-30&priceCur=KRW&textToSearch=%EC%84%9C%EC%9A%B8&travellerType=1&familyMode=off&ds=TIxA8l233xaGM%2Fhi&hotelStarRating=4&hotelAccom=34&productType=-1"
driver.get(url)
time.sleep(2)

#element = driver.find_element(By.CSS_SELECTOR,"Autocomplete_txt__nb6wT").click()
#titles = drive.find_element(By.CSS_SELECTOR, 'div.box_headline a.link_txt').text # (.)은 클래스를 나타냄
#print(titles)

# driver.quit()

actions = ActionChains(driver)
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    for _ in range (10):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height

hotels_raw = driver.find_elements(By.CLASS_NAME, "PropertyCard__Link")


hotels_url_list =[]
hotels_names_list=[]
hotels_location_list=[]
hotels_star_list=[]
hotels_rate_list=[]
for i in range(len(hotels_raw)):
    hotels_url_list.append(hotels_raw[i].get_attribute('href'))
    hotels_names_list.append(hotels_raw[i].get_attribute('aria-label'))
    print(hotels_raw[i].get_attribute('aria-label'),hotels_raw[i].get_attribute('href'))
    

인터시티 서울 호텔 (Intercity Seoul Hotel) https://www.agoda.com/ko-kr/intercity-seoul-hotel/hotel/seoul-kr.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1891463&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-12-29&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tag=45b17d1d-e0b0-fe2a-ce90-5513829d856b&isCityHaveAsq=false&tspTypes=7&los=1&searchrequestid=c446ef5a-e279-43fe-9200-33246c7f568c
코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park) https://www.agoda.com/ko-kr/courtyard-seoul-botanic-park_2/hotel/seoul-kr.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1891463&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-12-29&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tag=45b17d1d-e0b0-fe

In [17]:
hotels_names_list

['인터시티 서울 호텔 (Intercity Seoul Hotel)',
 '코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park)',
 '나인트리 프리미어 호텔 인사동 (Nine Tree Premier Hotel Insadong)',
 '유니언 호텔 (UNION HOTEL)',
 '포포인츠 바이 쉐라톤 서울 구로 (Four Points by Sheraton Seoul, Guro)',
 '오라카이 청계산 호텔 (Orakai Cheonggyesan Hotel)',
 '강남에 있는 숙소 (4성급) (4-star hotel in the Gangnam neighborhood)',
 '엠디 호텔 독산 (구. 스타즈 호텔 독산) (MD Hotel Doksan (Formely Staz Hotel Doksan))',
 '스탠포드 호텔 서울 (Stanford Hotel Seoul)',
 '리베라 호텔 (Riviera Hotel)',
 '호텔 페이토 삼성 (Hotel Peyto Samseong)',
 '삼정 호텔 (Hotel Samjung)',
 '오라카이 대학로 호텔 (Orakai Daehakro Hotel)',
 '이비스 스타일 앰배서더 서울 명동 (ibis Styles Ambassador Seoul Myeongdong)',
 '동서울 호텔 (Dong Seoul Hotel)',
 '신라스테이 구로 (Shilla Stay Guro)',
 '더 퍼스트 스테이 호텔 (The first stay hotel(old, Luce bridge hotel) (Korea Quality))',
 '신라스테이 마포 (Shilla Stay Mapo)',
 '아미드 호텔 서울 (Amid Hotel Seoul)',
 '코리아나 호텔 (Koreana Hotel)',
 '서울 가든 호텔 (Seoul Garden Hotel)',
 '신라스테이 서초 (Shilla Stay Seocho)',
 '이비스 스타일 앰배서더 서울 용산 (ibis Styles Ambassador Seoul Yon

In [18]:
hotels_location_list

[]

In [19]:
hotels_names_list

['인터시티 서울 호텔 (Intercity Seoul Hotel)',
 '코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park)',
 '나인트리 프리미어 호텔 인사동 (Nine Tree Premier Hotel Insadong)',
 '유니언 호텔 (UNION HOTEL)',
 '포포인츠 바이 쉐라톤 서울 구로 (Four Points by Sheraton Seoul, Guro)',
 '오라카이 청계산 호텔 (Orakai Cheonggyesan Hotel)',
 '강남에 있는 숙소 (4성급) (4-star hotel in the Gangnam neighborhood)',
 '엠디 호텔 독산 (구. 스타즈 호텔 독산) (MD Hotel Doksan (Formely Staz Hotel Doksan))',
 '스탠포드 호텔 서울 (Stanford Hotel Seoul)',
 '리베라 호텔 (Riviera Hotel)',
 '호텔 페이토 삼성 (Hotel Peyto Samseong)',
 '삼정 호텔 (Hotel Samjung)',
 '오라카이 대학로 호텔 (Orakai Daehakro Hotel)',
 '이비스 스타일 앰배서더 서울 명동 (ibis Styles Ambassador Seoul Myeongdong)',
 '동서울 호텔 (Dong Seoul Hotel)',
 '신라스테이 구로 (Shilla Stay Guro)',
 '더 퍼스트 스테이 호텔 (The first stay hotel(old, Luce bridge hotel) (Korea Quality))',
 '신라스테이 마포 (Shilla Stay Mapo)',
 '아미드 호텔 서울 (Amid Hotel Seoul)',
 '코리아나 호텔 (Koreana Hotel)',
 '서울 가든 호텔 (Seoul Garden Hotel)',
 '신라스테이 서초 (Shilla Stay Seocho)',
 '이비스 스타일 앰배서더 서울 용산 (ibis Styles Ambassador Seoul Yon

In [20]:
location = driver.find_elements(By.CSS_SELECTOR,"#property-main-content > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.iyCRmc > div.HeaderCerebrum > div.HeaderCerebrum__Location > span.Spanstyled__SpanStyled-sc-16tp9kb-0.gwICfd.kite-js-Span.HeaderCerebrum__Address")

In [21]:
df = pd.DataFrame({
    "hotel_name": hotels_names_list,
    "hotel_star":None,
    "hotel_rate":None,
    "hotel_url": hotels_url_list,
    "hotel_location": None,
    "review": None  # Initialize the "review" column with None
})

In [22]:
sc-jrAGrp sc-kEjbxe fzPhrN gOeEsn

UsageError: option -j not recognized ( allowed: "lv" )


In [23]:
rate = driver.find_elements(By.CLASS_NAME,"sc-jrAGrp sc-kEjbxe fzPhrN gOeEsn")

In [24]:
rate[0].text

IndexError: list index out of range

In [ ]:
star = driver.find_elements(By.CSS_SELECTOR,"#property-main-content > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.iyCRmc > div.HeaderCerebrum > div:nth-child(1) > div > span")

In [ ]:
star[0].get_attribute('aria-label')

IndexError: list index out of range

In [ ]:
Review-comment-bodyText,Review-comment-leftScore

NameError: name 'Review' is not defined

In [27]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException


stars = []
reviews = []

for i in range(len(hotels_url_list)):
    # 호텔 url 들어가기 
    driver.get(hotels_url_list[i])
    try:
        time.sleep(5)
        review_content_list = []
        reviwer_rate_list = []
        # 주소 가져오기
        location = driver.find_elements(By.CSS_SELECTOR, "#property-main-content > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.iyCRmc > div.HeaderCerebrum > div.HeaderCerebrum__Location > span.Spanstyled__SpanStyled-sc-16tp9kb-0.gwICfd.kite-js-Span.HeaderCerebrum__Address")
        
        # Check if location is not empty before accessing its elements
        if location:
            df.at[i, "hotel_location"] = location[0].text
        else:
            print(f"Location not found for URL {hotels_url_list[i]}")
            continue  # Skip to the next iteration if location is not found

        star = driver.find_elements(By.CSS_SELECTOR, "#property-main-content > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.iyCRmc > div.HeaderCerebrum > div:nth-child(1) > div > span")
        
        # Check if star is not empty before accessing its elements
        if star:
            df.at[i, "hotel_star"] = star[0].get_attribute('aria-label')
        else:
            print(f"Star not found for URL {hotels_url_list[i]}")
            # Set a default value or take appropriate action
            df.at[i, "hotel_star"] = None

        # 이용후기 누르기
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="hotelNavBar"]/div/ul/li[4]/button/span').click()
        time.sleep(2)

        page_counter = 0

        while page_counter < 1:
            reviwer_rate = driver.find_elements(By.CLASS_NAME, "Review-comment-leftScore")
            content = driver.find_elements(By.CLASS_NAME, "Review-comment-bodyText")
            for num in range(len(content)):
                review_content_list.append([reviwer_rate[num].text, content[num].text])
            try:
                # 페이지 넘기기
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewSection"]/div[4]/div/span[3]/button'))
                )
                ActionChains(driver).move_to_element(next_button).click().perform()
                time.sleep(1)
            except ElementClickInterceptedException:
                print(f"ElementClickInterceptedException: Next button is not clickable. Skipping to the next hotel.")
                break
            except TimeoutException:
                print("TimeoutException: Timed out waiting for the next button.")
                break
            page_counter += 1

        df.at[i, "review"] = review_content_list

        rate = driver.find_elements(By.CSS_SELECTOR, "#property-critical-root > div > div.Northstar > div.Box-sc-kv6pi1-0.cxnmtN > div.NorthstarSideContent > div.Box-sc-kv6pi1-0.cJiLOx.sc-gGmIRh.jjvpmH > div > div:nth-child(1) > div > div > div > div > div > div > p.sc-jrAGrp.sc-kEjbxe.fzPhrN.gOeEsn")
        
        # Check if rate is not empty before accessing its elements
        if rate:
            df.at[i, "hotel_rate"] = rate[0].text
        else:
            print(f"Rate not found for URL {hotels_url_list[i]}")
            # Set a default value or take appropriate action
            df.at[i, "hotel_rate"] = None

    except TimeoutException as e:
        print(f"TimeoutException occurred for URL {hotels_url_list[i]}: {e}")
        # Handle the exception, you might want to skip this URL or take appropriate action
    except ElementClickInterceptedException as e:
        print(f"ElementClickInterceptedException occurred for URL {hotels_url_list[i]}: {e}")
        # Handle the exception, you might want to skip this hotel or take appropriate action
    except NoSuchElementException as e:
        print(f"NoSuchElementException occurred for URL {hotels_url_list[i]}: {e}")
        # Handle the exception, you might want to skip this hotel or take appropriate action

# Close the WebDriver at the end
driver.quit()


TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
Location not found for URL https://www.agoda.com/ko-kr/Agoda-Secret-Deals/south-korea/seoul/38b120c.secret?finalPriceView=1&isShowMobileAppPrice=false&cid=1891463&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-12-29&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tag=45b17d1d-e0b0-fe2a-ce90-5513829d856b&isCityHaveAsq=false&los=1&searchrequestid=c446ef5a-e279-43fe-9200-33246c7f568c
TimeoutException: Timed out waiting for the next button.
TimeoutException: Timed out waiting for the next button.
Timeout

KeyboardInterrupt: 

In [26]:
df

,hotel_name,hotel_star,hotel_rate,hotel_url,hotel_location,review
0,인터시티 서울 호텔 (Intercity Seoul Hotel),None,None,https://www.agoda.com/ko-kr/intercity-seoul-ho...,None,None
1,코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park),None,None,https://www.agoda.com/ko-kr/courtyard-seoul-bo...,None,None
2,나인트리 프리미어 호텔 인사동 (Nine Tree Premier Hotel Insa...,None,None,https://www.agoda.com/ko-kr/nine-tree-premier-...,None,None
3,유니언 호텔 (UNION HOTEL),None,None,https://www.agoda.com/ko-kr/seoul-yeongdeungpo...,None,None
4,포포인츠 바이 쉐라톤 서울 구로 (Four Points by Sheraton Seo...,None,None,https://www.agoda.com/ko-kr/best-western-premi...,None,None
...,...,...,...,...,...,...
94,머큐어 앰배서더 서울 홍대 (Mercure Ambassador Seoul Hongdae),None,None,https://www.agoda.com/ko-kr/mercure-ambassador...,None,None
95,보코서울강남 (voco Seoul Gangnam),None,None,https://www.agoda.com/ko-kr/voco-seoul-gangnam...,None,None
96,L7 명동 (L7 Myeongdong),None,None,https://www.agoda.com/ko-kr/l7-myeongdong-by-l...,None,None
97,더 스테이 클래식 명동 (The Stay Classic Hotel Myeong dong),None,None,https://www.agoda.com/ko-kr/the-stay-classic-h...,None,None


In [ ]:
df.to_csv("./hotel_5star_agoda.csv",encoding="utf-8")
